In [14]:
# Uninstall the conflicting httpx version
# !pip uninstall -y httpx

In [47]:
# !pip install httpx==0.24.1
!pip install translate
!pip install gradio
!pip install googletrans==4.0.0-rc1
!pip install SpeechRecognition
!pip install --upgrade gradio
!pip install tensorflow-io
!pip install google-cloud-storage




# !pip install translate
# !pip install gradio
# !pip install googletrans==4.0.0-rc1


  Using cached httpx-0.27.0-py3-none-any.whl.metadata (7.2 kB)
  Using cached httpcore-1.0.5-py3-none-any.whl.metadata (20 kB)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
Using cached httpx-0.27.0-py3-none-any.whl (75 kB)
Using cached httpcore-1.0.5-py3-none-any.whl (77 kB)
Using cached h11-0.14.0-py3-none-any.whl (58 kB)
  Attempting uninstall: h11
    Found existing installation: h11 0.9.0
    Uninstalling h11-0.9.0:
      Successfully uninstalled h11-0.9.0
  Attempting uninstall: httpcore
    Found existing installation: httpcore 0.9.1
    Uninstalling httpcore-0.9.1:
      Successfully uninstalled httpcore-0.9.1
  Attempting uninstall: httpx
    Found existing installation: httpx 0.13.3
    Uninstalling httpx-0.13.3:
      Successfully uninstalled httpx-0.13.3


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
googletrans 4.0.0rc1 requires httpx==0.13.3, but you have httpx 0.27.0 which is incompatible.


  Using cached httpx-0.13.3-py3-none-any.whl.metadata (25 kB)
  Using cached httpcore-0.9.1-py3-none-any.whl.metadata (4.6 kB)
  Using cached h11-0.9.0-py2.py3-none-any.whl.metadata (8.1 kB)
Using cached httpx-0.13.3-py3-none-any.whl (55 kB)
Using cached httpcore-0.9.1-py3-none-any.whl (42 kB)
Using cached h11-0.9.0-py2.py3-none-any.whl (53 kB)
  Attempting uninstall: h11
    Found existing installation: h11 0.14.0
    Uninstalling h11-0.14.0:
      Successfully uninstalled h11-0.14.0
  Attempting uninstall: httpcore
    Found existing installation: httpcore 1.0.5
    Uninstalling httpcore-1.0.5:
      Successfully uninstalled httpcore-1.0.5
  Attempting uninstall: httpx
    Found existing installation: httpx 0.27.0
    Uninstalling httpx-0.27.0:
      Successfully uninstalled httpx-0.27.0


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradio 4.42.0 requires httpx>=0.24.1, but you have httpx 0.13.3 which is incompatible.
gradio-client 1.3.0 requires httpx>=0.24.1, but you have httpx 0.13.3 which is incompatible.
openai 1.40.2 requires httpx<1,>=0.23.0, but you have httpx 0.13.3 which is incompatible.


  Using cached httpx-0.27.0-py3-none-any.whl.metadata (7.2 kB)
  Using cached httpcore-1.0.5-py3-none-any.whl.metadata (20 kB)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
Using cached httpx-0.27.0-py3-none-any.whl (75 kB)
Using cached httpcore-1.0.5-py3-none-any.whl (77 kB)
Using cached h11-0.14.0-py3-none-any.whl (58 kB)
  Attempting uninstall: h11
    Found existing installation: h11 0.9.0
    Uninstalling h11-0.9.0:
      Successfully uninstalled h11-0.9.0
  Attempting uninstall: httpcore
    Found existing installation: httpcore 0.9.1
    Uninstalling httpcore-0.9.1:
      Successfully uninstalled httpcore-0.9.1
  Attempting uninstall: httpx
    Found existing installation: httpx 0.13.3
    Uninstalling httpx-0.13.3:
      Successfully uninstalled httpx-0.13.3


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
googletrans 4.0.0rc1 requires httpx==0.13.3, but you have httpx 0.27.0 which is incompatible.


In [51]:
import gradio as gr
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
from translate import Translator
import requests





In [52]:
# Initialize the Translator
def get_translator(language_code):
    return Translator(to_lang=language_code)

In [53]:
def translate_text(text, target_language):
    translator = Translator(to_lang=target_language)
    try:
        translation = translator.translate(text)
        return translation
    except Exception as e:
        return f"Translation Error: {e}"


In [55]:
def format_image_for_prediction(image):
    image = tf.image.resize(image, (300, 300))
    image_normalized = np.array(image).astype(np.float32) / 255
    image_normalized = np.expand_dims(image_normalized, axis=0)
    return image_normalized

In [65]:
# Function to get prediction from the model
def get_prediction(image_normalized, language):
    try:
#       gcs_model_path = 'gs://gh_gcs_bucket/melanoma_cnn_model.keras'
        gcs_model_path = 'melanoma_cnn_model.keras'
        model = tf.keras.models.load_model(gcs_model_path)
    except Exception as e:
        return translate_text(f"Error reading model: {str(e)}", language)

    try:
        prediction_probs = model.predict(image_normalized)
        prediction = np.argmax(prediction_probs, axis=1)
        confidence = np.max(prediction_probs)

        if prediction == 1:
            result = f"Our model's prediction is malignant with {confidence*100:.2f}% probability."
        else:
            result = f"Our model's prediction is benign with {confidence*100:.2f}% probability."

        return translate_text(result, language)
    except Exception as e:
        return translate_text(f"Prediction Error: {e}", language)



In [79]:
# def find_specialists(zip_code, language):
#     try:
#         api_url = f"https://npiregistry.cms.hhs.gov/api/?version=2.1&number=&enumeration_type=NPI-1&taxonomy_description=Oncology&postal_code={zip_code}&country_code=US&limit=5"

#         response = requests.get(api_url)

#         if response.status_code == 200:
#             data = response.json()
#             specialists = data.get('results', [])
#             if specialists:
#                 results = "\n".join([
#                     f"{spec.get('basic', {}).get('name', 'Name not found')} - "
#                     f"{spec.get('taxonomies', [{}])[0].get('desc', 'Specialization not found')}, "
#                     f"{spec.get('addresses', [{}])[0].get('address_1', 'Address not found')}, "
#                     f"{spec.get('addresses', [{}])[0].get('city', 'City not found')}, "
#                     f"{spec.get('addresses', [{}])[0].get('state', 'State not found')}"
#                     for spec in specialists
#                 ])
#                 return translate_text(f"Top cancer specialists near {zip_code}:\n{results}", language)
#             else:
#                 return translate_text(f"No cancer specialists found near {zip_code}.", language)
#         else:
#             return translate_text(f"Error retrieving data: {response.status_code}", language)
#     except Exception as e:
#         return translate_text(f"Error finding specialists: {e}", language)

In [83]:
def find_specialists(zip_code, language):
    try:
        api_url = f"https://npiregistry.cms.hhs.gov/api/?version=2.1&number=&enumeration_type=NPI-1&taxonomy_description=Oncology&postal_code={zip_code}&country_code=US&limit=5"
        response = requests.get(api_url)

        if response.status_code == 200:
            data = response.json()
            specialists = data.get('results', [])
            if specialists:
                results = []
                for spec in specialists:
                    # Extract the full name using first_name and last_name
                    first_name = spec.get('basic', {}).get('first_name', '')
                    last_name = spec.get('basic', {}).get('last_name', '')
                    name = f"{first_name} {last_name}".strip()
                    
                    # Extract the taxonomy
                    taxonomy_list = spec.get('taxonomies', [])
                    relevant_taxonomy = next(
                        (taxonomy.get('desc', 'Specialization not found') 
                         for taxonomy in taxonomy_list if 'Oncology' in taxonomy.get('desc', '')), 
                        None
                    )

                    if name and relevant_taxonomy:
                        # Format the result string
                        address = spec.get('addresses', [{}])[0].get('address_1', 'Address not found')
                        city = spec.get('addresses', [{}])[0].get('city', 'City not found')
                        state = spec.get('addresses', [{}])[0].get('state', 'State not found')
                        results.append(f"{name} - {relevant_taxonomy}, {address}, {city}, {state}")
                
                if results:
                    return translate_text(f"Top cancer specialists near {zip_code}:\n" + "\n".join(results), language)
                else:
                    return translate_text(f"No relevant oncology specialists found near {zip_code}.", language)
            else:
                return translate_text(f"No cancer specialists found near {zip_code}.", language)
        else:
            return translate_text(f"Error retrieving data: {response.status_code}", language)
    except Exception as e:
        return translate_text(f"Error finding specialists: {e}", language)

In [75]:
# Function to process image and get prediction and specialist info
def process_image(image, zip_code, language):
    image_normalized = format_image_for_prediction(image)
    prediction = get_prediction(image_normalized, language)
    specialists_info = find_specialists(zip_code, language)  # Assuming this function exists
    return prediction + "\n\n" + specialists_info

In [69]:
# Function to provide user instructions
def user_instructions(language):
    instructions = {
        "en": """
        **How to use the app:**
        1. Select your preferred language.
        2. Upload an image of the skin lesion.
        3. Enter your ZIP code to find nearby specialists.
        4. View the results, which include the prediction and nearby specialists.
        """,
        "es": """
        **Cómo usar la aplicación:**
        1. Seleccione su idioma preferido.
        2. Cargue una imagen de la lesión cutánea.
        3. Ingrese su código postal para encontrar especialistas cercanos.
        4. Vea los resultados, que incluyen la predicción y los especialistas cercanos.
        """,
        "fr": """   
        **Comment utiliser l'application:**
        1. Saisissez votre langue préférée.
        2. Chargez une image de la lèsion cutanée.
        3. Entrez votre code postal pour trouver des spécialistes à proximité.
        4. Consultez les résultats, qui incluent la prédiction et les spécialistes à proximité.
        """,
        "de": """
        **So verwenden Sie die App:**
        1. Wählen Sie Ihre bevorzugte Sprache aus.
        2. Laden Sie ein Bild der Hautläsion hoch.
        3. Geben Sie Ihre Postleitzahl ein, um nahegelegene Spezialisten zu finden.
        4. Sehen Sie sich die Ergebnisse an, die die Vorhersage und nahegelegene Spezialisten enthalten.
        """,
        "it": """
        **Come utilizzare l'app:**
        1. Seleziona la tua lingua preferita.
        2. Carica un'immagine della lesione cutanea.
        3. Inserisci il tuo codice postale per trovare specialisti nelle vicinanze.
        4. Visualizza i risultati, che includono la previsione e gli specialisti nelle vicinanze.
        """,
        "zh": """
        **如何使用该应用程序：**
        1. 选择您的首选语言。
        2. 上传皮肤病变的图像。
        3. 输入您的邮政编码以查找附近的专家。
        4. 查看结果，包括预测和附近的专家。
        """,
        "hi": """
        **ऐप का उपयोग कैसे करें:**
        1. अपनी पसंदीदा भाषा चुनें।
        2. त्वचा के घाव की एक छवि अपलोड करें।
        3. निकटवर्ती विशेषज्ञों को खोजने के लिए अपना ज़िप कोड दर्ज करें।
        4. परिणाम देखें, जिसमें पूर्वानुमान और निकटवर्ती विशेषज्ञ शामिल हैं।
        """,
        "ar": """
        **كيفية استخدام التطبيق:**
        1. اختر لغتك المفضلة.
        2. تحميل صورة للآفة الجلدية.
        3. أدخل الرمز البريدي الخاص بك للعثور على المتخصصين القريبين.
        4. عرض النتائج التي تشمل التنبؤ والمتخصصين القريبين.
        """,
        "bn": """
        **অ্যাপটি কীভাবে ব্যবহার করবেন:**
        1. আপনার পছন্দের ভাষা নির্বাচন করুন।
        2. ত্বকের ক্ষতের একটি ছবি আপলোড করুন।
        3. নিকটবর্তী বিশেষজ্ঞদের খুঁজে পেতে আপনার ZIP কোড লিখুন।
        4. ফলাফলগুলি দেখুন, যাতে পূর্বাভাস এবং নিকটবর্তী বিশেষজ্ঞরা অন্তর্ভুক্ত রয়েছে।
        """,
        "pt": """   
        **Como usar o aplicativo:**
        1. Selecione seu idioma preferido.
        2. Carregue uma imagem da lesão cutânea.
        3. Insira seu código postal para encontrar especialistas próximos.
        4. Veja os resultados, que incluem a previsão e os especialistas próximos.
        """,
        "ru": """
        **Как использовать приложение:**
        1. Выберите предпочитаемый язык.
        2. Загрузите изображение поражения кожи.
        3. Введите свой почтовый индекс, чтобы найти ближайших специалистов.
        4. Просмотрите результаты, которые включают прогноз и ближайших специалистов.
        """,
        "ja": """
        **アプリの使い方：**
        1. 希望の言語を選択します。
        2. 皮膚病変の画像をアップロードします。
        3. 近くの専門家を見つけるために郵便番号を入力します。
        4. 予測と近くの専門家を含む結果を表示します。
        """,
}
    return instructions.get(language, instructions["en"])

# Function to update instructions
def update_instructions(language):
    return user_instructions(language)


In [10]:
# # FAQ content
# faq_content = """
# ### FAQs:
# - **What is melanoma?**  
#   Melanoma is a serious type of skin cancer caused by uncontrolled growth of pigment-producing cells.

# - **Is melanoma a serious disease?**  
#   Yes, it can spread to other organs and be fatal if not detected early.

# - **What causes melanoma?**  
#   Excessive sun exposure is the most common cause, along with genetic factors.

# - **What does melanoma look like?**  
#   It often starts as a flat, unevenly bordered marking.

# - **Can melanoma be cured?**  
#   Yes, if detected early, it can often be cured through surgical removal.

# - **What are the risk factors?**  
#   Fair skin, a history of sunburns, moles, and a family history of skin cancer are key risk factors.

# - **Can melanoma be prevented?**  
#   Yes, by avoiding peak sunlight hours, using sunscreen, and wearing protective clothing.

# - **Current melanoma research:**  
#   Studies focus on antioxidants like lycopene and flavonoids for prevention.
# """

# # Resources content
# resources_content = """
# ### Resources:
# - **Melanoma Survival Rates:** [Click Here](https://www.curemelanoma.org/about-melanoma/melanoma-staging/melanoma-survival-rates)
# - **Questions to Ask Your Doctor:** [Click Here](https://www.cancer.org/cancer/types/melanoma-skin-cancer/detection-diagnosis-staging/talking-with-doctor.html)
# - **Melanoma Skin Cancer Information:** [Click Here](https://www.cancer.org/cancer/types/melanoma-skin-cancer.html)
# - **What is Melanoma:** [Click Here](https://www.cancer.org/cancer/types/melanoma-skin-cancer/about/what-is-melanoma.html)
# """

In [17]:
# # Update the interface description based on language selection
# def update_instructions(language):
#     return user_instructions(language)


In [60]:
# Custom CSS styling for the app
custom_css = """
body {
    background-color: #ffffff;  /* White Background */
    font-family: 'Arial', sans-serif;  /* Simple font */
    color: #000000;  /* Black text color */
}

h1, h2 {
    color: #000000;  /* Black heading color */
    text-align: center;
    font-weight: bold;  /* Bold heading */
}

.gradio-container {
    max-width: 800px;
    margin: auto;
    padding: 20px;
    background-color: #f5f5f5;  /* Light grey for the container */
    border-radius: 15px;
    box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);
}

button {
    background-color: #e0e0e0 !important;  /* Light Gray Buttons */
    color: #000000 !important;  /* Black text color */
    border-radius: 5px;
    padding: 10px 20px;
    font-weight: bold;  /* Bold button text */
}

textarea, input, .gr-box, .gr-file-upload {
    border-radius: 5px;
    padding: 10px;
    font-weight: bold;  /* Bold input and textarea text */
    color: #000000;  /* Black font color */
    background-color: #ffffff !important;  /* White input background */
    border: 2px solid #333333;  /* Dark gray border for the inputs */
}

.output-textbox {
    background-color: #f5f5f5;  /* Light grey for the output box */
    color: #000000;  /* Black font color */
    border: 3px solid #333333;  /* Dark gray border */
    padding: 10px;
    font-weight: bold;  /* Bold output text */
    border-radius: 10px;
    box-shadow: 0 4px 8px rgba(0, 0, 0, 0.2);  /* Subtle shadow for the output box */
}
"""


In [70]:
# Creating Gradio interface
interface = gr.Blocks()

with interface:
    with gr.Row():
        with gr.Column():
            language = gr.Dropdown(
                choices=["en", "es", "zh", "hi", "ar", "bn", "pt", "ru", "ja", "de", "fr"],
                label="Select Language"
            )
            instructions = gr.Markdown(user_instructions("en"))  # Default to English instructions
            language.change(fn=user_instructions, inputs=language, outputs=instructions)

            # Image component that will both accept and display the uploaded image
            image = gr.Image(type="pil", label="Upload a Skin Lesion Image", interactive=True)
            zip_code = gr.Textbox(label="Enter Your Zip Code")
            submit_button = gr.Button("Submit")

            output = gr.Textbox(label="Prediction and Specialists", lines=10)

            submit_button.click(
                fn=process_image,
                inputs=[image, zip_code, language],
                outputs=output
            )

    with gr.Row():
        faq = gr.Markdown(faq_content)
        resources = gr.Markdown(resources_content)


In [84]:
# Launch the app
interface.launch(share=True)


Rerunning server... use `close()` to stop if you need to change `launch()` parameters.
----
Running on public URL: https://7cd1927aa0bcb45381.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 417ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 379ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 380ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 395ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 414ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 440ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 395ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 370ms/step
